In [6]:
import numpy as np
import torch
from robust_gcn.robust_gcn import sparse_tensor
from sacred import Experiment
from scipy.sparse import csr_matrix
from tqdm import tqdm

from robust_gcn.robust_gcn import RobustGCNModel, certify
from equivariance_robustness.attributes_zuegner.training import train
from equivariance_robustness.data import (load_node_classification_dataset,
                                          split)
from equivariance_robustness.utils import set_seed

In [7]:
dataset_params = {
    'name': 'Cora',
    'root': 'your/path/to/datasets/planetoid',
    'n_per_class': 20
}

training_params = {
    'n_iters': 3000,
    'method': 'Normal',
    'learning_rate': 1e-3,
    'weight_decay': 5e-4,
    'early_stopping': 50
}

certification_params = {
    'q': 0.01,
    'q_relative': True,
    'Q_max': 50,
    'optimize_omega': False,
    'optimize_steps': 5,
    'batch_size': 8,
    'certify_nonrobustness': False,
    'cost_add': 1,
    'cost_del': 1
}

hidden_sizes = [32]

seed = 0

In [8]:
set_seed(seed)

if torch.cuda.is_available():
        device = torch.device('cuda:0')
else:
    device = torch.device('cpu')


In [9]:
data = load_node_classification_dataset(device=torch.device('cpu'), seed=seed, root=dataset_params['root'], name=dataset_params['name'])

In [10]:
edge_idx = data.edge_index.numpy()
X = data.x.numpy()
y = data.y.numpy().astype('int8')
A = csr_matrix((np.ones(len(edge_idx[0])), (edge_idx[0], edge_idx[1])))
X = csr_matrix(X)
K = y.max()+1
N,D = X.shape
X_t = sparse_tensor(X).to(device)
y_t = torch.tensor(y.astype("int64"), device=device)

In [11]:
# Split data
idx_train, idx_val, idx_test = split(y, n_per_class=20)

# Create model
model = RobustGCNModel(A, [D]+hidden_sizes+[K]).cuda()

# Train model
q = certification_params['q']
if certification_params['q_relative']:
    q = q * D
q = int(q)

In [12]:
train(gcn_model=model, X=X_t, y=y_t, idx_train=idx_train, idx_val=idx_val)

  4%|▍         | 119/3000 [00:01<00:19, 150.67it/s]

In [ ]:
model.eval()
pred = model.predict(X_t.to_dense(), np.arange(N)).detach().cpu().numpy()

In [ ]:
results_dict = {
        'targets': y,
        'pred': pred,
        'idx_train': idx_train,
        'idx_val': idx_val,
        'idx_test': idx_test,
        'test_accuracy': (pred[idx_test] == y[idx_test]).mean(),
    }

In [ ]:
certified = []
attacked = []
max_rad = 0

In [ ]:
for Q in tqdm(range(1, certification_params['Q_max']+1)):
    certified_at_Q, attacked_at_Q = certify(
        gcn_model=model, attrs=X.astype("float32"),
        q=q, Q=Q,
        optimize_omega=certification_params['optimize_omega'],
        optimize_steps=certification_params['optimize_steps'],
        batch_size=certification_params['batch_size'],
        certify_nonrobustness=True,
        progress=False)

    certified_at_Q, attacked_at_Q = certified_at_Q.astype(bool), attacked_at_Q.astype(bool)

    certified.append(certified_at_Q)
    attacked.append(attacked_at_Q)

    if np.sum(attacked_at_Q) == N:
        break
    else:
        max_rad += 1

100%|██████████| 50/50 [15:11<00:00, 18.23s/it]


In [ ]:
certified = np.vstack(certified)
attacked = np.vstack(attacked)

In [ ]:
results_dict.update({
    'certified': certified,
    'attacked': attacked,
    'max_rad': max_rad
})

In [ ]:
torch.save(results_dict,
           './results')